In [1]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython


Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
%cd /content/
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/tensorflow/tensorflow.git
import os

# and import it.
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

os.environ['PYTHONPATH'] += ":/content/models:/content/models/research:/content/models/research/slim"
!python setup.py build
!python setup.py install


/content
Cloning into 'models'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 33643 (delta 23), reused 43 (delta 6), pack-reused 33565
Receiving objects: 100% (33643/33643), 512.05 MiB | 35.35 MiB/s, done.
Resolving deltas: 100% (21675/21675), done.
Checking out files: 100% (2486/2486), done.
Cloning into 'tensorflow'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 845306 (delta 72), reused 143 (delta 24), pack-reused 845108
Receiving objects: 100% (845306/845306), 489.76 MiB | 28.72 MiB/s, done.
Resolving deltas: 100% (684779/684779), done.
Checking out files: 100% (19701/19701), done.
/content/models/research
running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> buil

In [14]:
%cd /content/drive
%cd "/content/drive/My Drive/Colab Notebooks/CollisionDetection"
%ls

/content/drive
/content/drive/My Drive/Colab Notebooks/CollisionDetection
images/                                  training/
ssd_inception_v2_coco_2018_01_28/        Untitled1.ipynb
ssd_inception_v2_coco_2018_01_28.tar.gz


In [6]:
# using ssd_inception_v2_coco	
!curl http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz > ssd_inception_v2_coco_2018_01_28.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  265M  100  265M    0     0  81.2M      0  0:00:03  0:00:03 --:--:-- 81.3M
tar: ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [7]:
# untar
!tar -xvf ssd_inception_v2_coco_2018_01_28.tar.gz

ssd_inception_v2_coco_2018_01_28/
ssd_inception_v2_coco_2018_01_28/model.ckpt.index
ssd_inception_v2_coco_2018_01_28/checkpoint
ssd_inception_v2_coco_2018_01_28/pipeline.config
ssd_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
ssd_inception_v2_coco_2018_01_28/model.ckpt.meta
ssd_inception_v2_coco_2018_01_28/saved_model/
ssd_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
ssd_inception_v2_coco_2018_01_28/saved_model/variables/
ssd_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [11]:
# config files

!curl https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/ssd_inception_v2_coco.config > training/ssd_inception_v2_coco.config

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4646  100  4646    0     0  23230      0 --:--:-- --:--:-- --:--:-- 23230


In [0]:
import os 
import cv2 
import numpy as np 
import tensorflow as tf 
import sys 
%cd /content/models/research/
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util 

# Name of the directory containing the object detection module we're using 
MODEL_NAME = 'ssd_inception_v2_coco_2018_01_28' # The path to the directory where frozen_inference_graph is stored. 
IMAGE_NAME = 'images/img375.png' # The path to the image in which the object has to be detected. 

# Grab path to current working directory 
CWD_PATH = "/content/drive/My Drive/Colab Notebooks/CollisionDetection" #os.getcwd() 

# Path to frozen detection graph .pb file, which contains the model that is used 
# for object detection. 
PATH_TO_CKPT = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb') 

# Path to label map file 
PATH_TO_LABELS = os.path.join(CWD_PATH, 'training', 'labelmap.pbtxt') 

# Path to image 
PATH_TO_IMAGE = os.path.join(CWD_PATH, 'images', IMAGE_NAME) 

# Number of classes the object detector can identify 
NUM_CLASSES = 1

# Load the label map. 
# Label maps map indices to category names, so that when our convolution 
# network predicts `5`, we know that this corresponds to `king`. 
# Here we use internal utility functions, but anything that returns a 
# dictionary mapping integers to appropriate string labels would be fine 
label_map = label_map_util.load_labelmap(PATH_TO_LABELS) 
categories = label_map_util.convert_label_map_to_categories( 
		label_map, max_num_classes = NUM_CLASSES, use_display_name = True) 
category_index = label_map_util.create_category_index(categories) 

# Load the Tensorflow model into memory. 
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
	od_graph_def = tf.GraphDef() 
	with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
		serialized_graph = fid.read() 
		od_graph_def.ParseFromString(serialized_graph) 
		tf.import_graph_def(od_graph_def, name ='') 

	sess = tf.Session(graph = detection_graph) 

# Define input and output tensors (i.e. data) for the object detection classifier 

# Input tensor is the image 
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 

# Output tensors are the detection boxes, scores, and classes 
# Each box represents a part of the image where a particular object was detected 
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 

# Each score represents level of confidence for each of the objects. 
# The score is shown on the result image, together with the class label. 
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 

# Number of objects detected 
num_detections = detection_graph.get_tensor_by_name('num_detections:0') 

# Load image using OpenCV and 
# expand image dimensions to have shape: [1, None, None, 3] 
# i.e. a single-column array, where each item in the column has the pixel RGB value 
image = cv2.imread(PATH_TO_IMAGE) 

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 25 # Brightness control (0-100)
# image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

image_expanded = np.expand_dims(image, axis = 0) 

# Perform the actual detection by running the model with the image as input 
(boxes, scores, classes, num) = sess.run( 
	[detection_boxes, detection_scores, detection_classes, num_detections], 
	feed_dict ={image_tensor: image_expanded}) 

# Draw the results of the detection (aka 'visualize the results') 
# print(classes)

vis_util.visualize_boxes_and_labels_on_image_array( 
	image, 
	np.squeeze(boxes), 
	np.squeeze(classes).astype(np.int32), 
	np.squeeze(scores), 
	category_index, 
	use_normalized_coordinates = True, 
	line_thickness = 2, 
	max_boxes_to_draw= 10, #https://stats.stackexchange.com/questions/297796/faster-r-cnn-how-to-avoid-multiple-detection-in-same-area
	min_score_thresh = 0.0) 

from google.colab.patches import cv2_imshow
%matplotlib inline
# All the results have been drawn on the image. Now display the image. 
cv2_imshow(image) 

# Press any key to close the image 
cv2.waitKey(0) 

# Clean up 
cv2.destroyAllWindows()
